In [1]:
clean_up = True # if True, remove all gams related files from working folder before starting
%run stdPackages.ipynb

The file _gams_py_gdb0.gdx is still active and was not deleted.


*Load specific modules used here:*

In [2]:
os.chdir(d['py'])
import mProduction
from gmsPython import nestingTree
from valueShares import nestedShares

# Production modules

Load data:

In [3]:
t0 = 2019
v = 'vSmall'
db = GpyDB(os.path.join(d['data'], f'{v}{t0}_db'), name = f'{v}{t0}_db', ws = d['work'])
ws = db.ws

For variables that are defined over $t$, but where we do not yet have an initial value for all $t$, extrapolate from data:

*Note: This forces extrapolation of all variables defined over $t$ - if it is important that some variables are not extrapolated, they should be removed from this statement.*

In [4]:
[symbol.__setattr__('vals', extrapolateUpper(symbol.vals, db('tE')[0])) for symbol in [db[k] for k in db.varDom('t')['t']]];

Subset data to only include production sectors, but keep a full copy:

In [5]:
db0 = db.copy()
AggDB.subsetDB(db, db('s_p'))

## 1. ```StaticNCES```

### 1.1. Default implementation

In [6]:
db_StaticNCES = db.copy()
db_i = db_StaticNCES

Remove durables (and their value from supply side):

In [7]:
db_i.aom(adj.rc_pd(db_i('qS'), db_i('n_p'))-(adj.rc_pd(db_i('qD'), db_i('dur_p')) * db_i('pD_dur')).groupby(['t','s']).sum(), name = 'qS', priority='second')
db_i['qD'] = adj.rc_pd(db_i('qD'), ('not', db_i('dur_p')))
db_i['nestProdInp'] = adj.rc_pd(db_i('nestProdInp'), ('not', db_i('dur_p').rename('nn')))

*Set up nesting structure from nesting tree:*

In [8]:
nestInp = nestingTree.Tree('Inp', tree = db_i('nestProdInp').to_list(), f = 'CES')
nestAgg = nestingTree.AggTree(name = 'P', trees = {t.name: t for t in [nestInp]}, ws = ws)
nestAgg(namespace = {str(n)+'_input':n for n in db('n')})

*Get starting values for $\mu$ using value shares:*

In [9]:
v = nestedShares(nestAgg, ws = ws)
db_p = v(db)

Use share parameters and values to get initial values for intermediates:

In [10]:
db_i.aom(db_p('mu').xs(t0), name = 'mu')
db_i.aom(adj.rc_pd(db_p('vD'), nestAgg.get('int')).rename('qD'), name = 'qD', priority = 'first') # specify intermediate goods levels

Initialize model instance:

In [11]:
m_StaticNCES = mProduction.getStaticNCES(nestAgg, partial = True, initFromGms = 'initFirmValueBlock')
m_i = m_StaticNCES

Add data and initialize other stuff with default options:

In [12]:
m_i.initStuff(db = db_i)

Solve calibration with robust relaxation method:

In [13]:
solve_StaticNCES_Calib = m_i.jSolve(10, state = 'C')
solve_i = solve_StaticNCES_Calib

Add solution to main database, remove init methods (we want to use this solution as initial value):

In [14]:
[m_i.db.__setitem__(k, solve_i[k]) for k in m_i.db.getTypes(['var']) if k in solve_i.symbols]; # use solution database
m_i.db.mergeInternal()
m_i.initFromGms = None # no longer use gams code to initialize variables

Test baseline state:

In [15]:
solve_StaticNCES = m_i.solve(state = 'B') # solve baseline state
solve_i = solve_StaticNCES

### 1.2. Simple emission extension:

Initialize emission extension:

In [16]:
# m_StaticNCES_emission = mProduction.StaticNCES_emission(nestAgg, partial = True, abateCosts = False) # Initialize directly from extension class
m_StaticNCES_emission = mProduction.getStaticNCES(nestAgg, extension = 'emission', partial = True, abateCosts = False) # ignore abatement costs
m_i = m_StaticNCES_emission

Test run this as well:

In [17]:
m_i.initStuff(db = db_i) # don't need to add more data, we already rely on the solution from the basic model version here.
solve_StaticNCES_emission = m_i.solve(state = 'C')

Now, test run with some type of abatement costs:

In [18]:
m_i.addAbatementCosts("SqrAdjCosts")
m_i.initStuff() # update group definitions and such
solve_StaticNCES_emissionWithAbate = m_i.solve(state = 'C')

## Dynamic  

The dynamic classes currently only contains the same extensions as the static models. Technically, extensions are caught by the parent class, thus both instances here are simple ```DynamicNCES_base```.

### 2.1. Without emissions (default)

In [19]:
db_DynamicNCES = db.copy()
db_i = db_DynamicNCES

*Set up nesting structure from nesting tree:*

In [20]:
nestInp = nestingTree.Tree('Inp', tree = db_i('nestProdInp').to_list(), f = 'CES')
nestAgg = nestingTree.AggTree(name = 'P', trees = {t.name: t for t in [nestInp]}, ws = ws)
nestAgg(namespace = {str(n)+'_input':n for n in db('n')})

*Get starting values for $\mu$ using value shares:*

In [21]:
v = nestedShares(nestAgg, ws = ws)
db_p = v(db)

Use share parameters and values to get initial values for intermediates:

In [22]:
db_i.aom(db_p('mu').xs(t0), name = 'mu')
db_i.aom(adj.rc_pd(db_p('vD'), nestAgg.get('int')).rename('qD'), name = 'qD', priority = 'first') # specify intermediate goods levels

Use static user cost as initial guess for price on durables (if no other has been provided):

In [23]:
db_i.aom(db('pD_dur'), name = 'pD', priority = 'first') # pD_dur includes an estimate of the long-run user-cost of capital

Initialize model instance:

In [24]:
m_DynamicNCES = mProduction.getDynamicNCES(nestAgg, partial = True, initFromGms = 'initFirmValueBlock')
m_i = m_DynamicNCES

Add data and initialize other stuff with default options:

In [25]:
m_i.initStuff(db = db_i)

Solve calibration with robust relaxation method:

In [26]:
solve_DynamicNCES_Calib = m_i.jSolve(10, state = 'C')
solve_i = solve_DynamicNCES_Calib

Add solution to main database, remove init methods (we want to use this solution as initial value):

In [27]:
[m_i.db.__setitem__(k, solve_i[k]) for k in m_i.db.getTypes(['var']) if k in solve_i.symbols]; # use solution database
m_i.db.mergeInternal()
m_i.initFromGms = None # no longer use gams code to initialize variables

Test baseline state:

In [28]:
solve_DynamicNCES = m_i.solve(state = 'B') # solve baseline state
solve_i = solve_DynamicNCES